In [1]:
import numpy as np
import pandas as pd
import os
import re
import string
from nltk.tokenize import word_tokenize
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import nltk
from nltk import word_tokenize
nltk.downloader
nltk.download('punkt')
from nltk import punkt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#SENT_DETECTOR = nltk.data.load('tokenizers/punkt/english.pickle')

In [4]:
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/ACOTAR.zip.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")

In [5]:
a="/content/ACOTAR"  

In [6]:
def stories(a):
    txt=[]
    for root, dirs, files in os.walk(a):
        for name in files:
            #print(os.path.join(root, name))
            with open(os.path.join(root, name),'r') as f:
             for line in f.readlines():
                    line = line.strip()
                    if line=='----------': break
                    if line!='':txt.append(line)
    return txt
stories = stories(a)
print("number of lines = ", len(stories)end=' ')


number of lines =  32779


In [7]:
def clean_txt(txt):
    cleaned_txt = []
    for line in txt:
        line = line.lower()
        line = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-\\]", "", line)
        tokens = word_tokenize(line)
        words = [word for word in tokens if word.isalpha()]
        cleaned_txt+=words
    return cleaned_txt

cleaned_stories = clean_txt(stories)
print("number of words = ", len(cleaned_stories),end=' ')

number of words =  330943 

In [8]:
print(cleaned_stories,end=' ')

Output hidden; open in https://colab.research.google.com to view.

In [9]:
def make_markov_model(cleaned_stories, n_gram=2):
    markov_model = {}
    for i in range(len(cleaned_stories)-n_gram-1):
        curr_state, next_state = "", ""
        for j in range(n_gram):
            curr_state += cleaned_stories[i+j] + " "
            next_state += cleaned_stories[i+j+n_gram] + " "
        curr_state = curr_state[:-1]
        next_state = next_state[:-1]
        if curr_state not in markov_model:
            markov_model[curr_state] = {}
            markov_model[curr_state][next_state] = 1
        else:
            if next_state in markov_model[curr_state]:
                markov_model[curr_state][next_state] += 1
            else:
                markov_model[curr_state][next_state] = 1
    
    # calculating transition probabilities
    for curr_state, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[curr_state][state] = count/total
        
    return markov_model

In [10]:
markov_model = make_markov_model(cleaned_stories)

In [13]:
print(markov_model)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print("number of states = ", len(markov_model.keys()))

number of states =  127515


In [ ]:
print("All possible transitions from 'high lord' state: \n")
print(markov_model['high lord'])

All possible transitions from 'high lord' state: 

{'then a': 0.00411522633744856, 'or if': 0.00411522633744856, 'who had': 0.01646090534979424, 's she': 0.00411522633744856, 'of the': 0.13168724279835392, 'click click': 0.00411522633744856, 'of one': 0.00411522633744856, 'of prythian': 0.02880658436213992, 'all of': 0.00411522633744856, 'i tucked': 0.00411522633744856, 'human the': 0.00411522633744856, 'does not': 0.00411522633744856, 'for you': 0.00411522633744856, 's manor': 0.00411522633744856, 'and live': 0.00411522633744856, 's side': 0.00823045267489712, 'high lord': 0.00823045267489712, 'feral rage': 0.00411522633744856, 'i should': 0.00411522633744856, 'enough but': 0.00411522633744856, 'at my': 0.00411522633744856, 'and everything': 0.00411522633744856, 'that wasn': 0.00411522633744856, 'be damned': 0.00411522633744856, 'to glamour': 0.00411522633744856, 'carried him': 0.00411522633744856, 'digging a': 0.00411522633744856, 'ah the': 0.00411522633744856, 'i shook': 0.004115226

In [ ]:
def generate_story(markov_model, limit=100, start='sherlock holmes'):
    n = 0
    curr_state = start
    next_state = None
    story = ""
    story+=curr_state+" "
    while n<limit:
        next_state = random.choices(list(markov_model[curr_state].keys()),
                                    list(markov_model[curr_state].values()))
        
        curr_state = next_state[0]
        story+=curr_state+" "
        n+=1
    return story

In [ ]:
for i in range(20):
    print(str(i)+". ", generate_story(markov_model, start="high lord", limit=8))

0.  high lord of the parlor i got rid of it or would not speak clotho gestured to her 
1.  high lord of the house none of the treaty did not demand we provide for our sister s 
2.  high lord whom i might spill this secret vulnerability a mercenary told me she went on these lands 
3.  high lord the hallways were bustling with masked faeries i was about to move past the first step 
4.  high lord of one of the sacrifice was borne by high fae long enough to see the continent 
5.  high lord s side and thesan you didn t stop him as i stepped so close that i 
6.  high lord and it didn t so much as looking toward him another blink they sold to some 
7.  high lord of prythian as well my father and nesta with little trade little opportunity hybern s people 
8.  high lord for a moment longer tying off the tapestry you had two suggestions based on azriel s 
9.  high lord of winter at few other members of their lives like shards in a broken sound of 
10.  high lord of the table with a thump it s an

In [ ]:
for i in markov_model:
  print(generate_story(markov_model, start="high lord", limit=100))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print(generate_story(markov_model, start="if rhysand", limit=100))

if rhysand was yelling it if it hurts you i bristled at the name echoing with the rustle of cassian s brutal training clicked into place beside me despite the horse s hooves was too loud on the information you need to order new ones she usually beneath her many of these paintings i hadn t realized i was gone the space where he hid miryam and drakon have you as my own skill set and sealed in any splinters so i told him they d touch my hand to reach her eyes tamlin must have been in his late twenties but he wasn t subtle with dagdan and brannagh accepted the outstretched silver cup they each barely had a idea their cruel ethereal faces and wide grins told me what happened with the high lord s seventh son but helion s face beron is a threat in that regard too so use me as bait i blinked the air was warm and mild and a low growl from a block away the scent that hit the it was a shooting star through the trees the forest it was another way out the pit loomed and with a beautiful priestess b

In [ ]:
print(generate_story(markov_model, start="high lord", limit=100))

high lord of day unleashed themselves upon hybern until they got hungry again but i jerked my chin between his he lurched bending over with a click that sounded through me he looked to mor but if beron or eris s as a soldier his other for mor then mor extended her hands in mine the bond between us but you want to know nuan answered tightly helion demanded of beron what does this come from rhys for over five hundred years you still mourned your commoner lover after all i tried to maintain that illusion of strength tamlin and my heart s wild pounding in my ears and i shoved it behind her with a sort of sound that might have allowed myself to dream that outrageously but i did it to save me keep me from the demons in this army camp bustling toward the horizon across the sea how she calls to you when hybern s forces for nothing this battle myself he waved a hand and did not speak of them in my painting as if he saw first stumbling out of the cottage well i said to graysen and his father a l